### This notebook is intended to predict sql injection attacks
    Total data  : 5234
    Benign data : 3072 + 744 = 3816
    SQLI Data   : 1128 + 290 = 1418
    
#### At the end of notebook are results of testing

In [24]:

# import required packages

import glob
import time
import pandas as pd
# from xml.dom import minidom
from nltk import ngrams
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import PorterStemmer
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to C:\Users\Acha
[nltk_data]     Prawira\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Acha
[nltk_data]     Prawira\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Acha
[nltk_data]     Prawira\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [25]:
import keras

In [26]:
import nltk
import pandas as pd
import os

### File_name: git_foospidy_payloads_fuzzing_code_db_sqli.txt
##### Comments :  
1. removed \n at the end of each line
2.  replaced %20 by space
#### Source: https://github.com/foospidy/payloads/blob/7749dd0bfcc64786feb22f578b978d2f8607e353/owasp/fuzzing_code_database/sqli/sqli.txt

In [27]:
# preprocess sql data to have same format for all files

def clean_sqli_data(data):
    
    for i in range(len(data)):
        
        data[i]=data[i].replace('\n', '')
        data[i]=data[i].replace('%20', ' ')
        data[i]=data[i].replace('=', ' = ')
        data[i]=data[i].replace('((', ' (( ')
        data[i]=data[i].replace('))', ' )) ')
        data[i]=data[i].replace('(', ' ( ')
        data[i]=data[i].replace(')', ' ) ')
    
    return data

In [28]:
path='./sqli.txt'

In [29]:
# read data from file

sql_lines_fuzzing=[]
f = open(path, "r")
for x in f:
    sql_lines_fuzzing.append(x)

In [30]:
sql_lines_fuzzing=clean_sqli_data(sql_lines_fuzzing) 

In [31]:
sql_lines_fuzzing[:15]

["'sqlvuln",
 "'+sqlvuln",
 'sqlvuln;',
 ' ( sqlvuln ) ',
 "a' or 1 = 1--",
 '"a"" or 1 = 1--"',
 ' or a  =  a',
 "a' or 'a'  =  'a",
 '1 or 1 = 1',
 "a' waitfor delay '0:0:10'--",
 "1 waitfor delay '0:0:10'--",
 'declare @q nvarchar  ( 4000 )  select @q  = ',
 '0x770061006900740066006F0072002000640065006C00610079002000270030003A0030003A',
 '0',
 '031003000270000']

### File_Name: git_foospidy_payloads_other_sqli_camoufl4g3.txt
##### Comments : ok
#### Source: https://github.com/foospidy/payloads/tree/master/other/sqli

In [32]:
path='./camoufl4g3.txt'

In [33]:
# read data from file

sql_lines_camoufl4g3=[]
f = open(path, "r")
for x in f:
    sql_lines_camoufl4g3.append(x)

In [34]:
sql_lines_camoufl4g3[:15] # data before cleaning

["'-'\n",
 "' '\n",
 "'&'\n",
 "'^'\n",
 "'*'\n",
 "' or ''-'\n",
 "' or '' '\n",
 "' or ''&'\n",
 "' or ''^'\n",
 "' or ''*'\n",
 '"-"\n',
 '" "\n',
 '"&"\n',
 '"^"\n',
 '"*"\n']

In [35]:
sql_lines_camoufl4g3=clean_sqli_data(sql_lines_camoufl4g3)

In [36]:
sql_lines_camoufl4g3[:15]  # data after cleaning

["'-'",
 "' '",
 "'&'",
 "'^'",
 "'*'",
 "' or ''-'",
 "' or '' '",
 "' or ''&'",
 "' or ''^'",
 "' or ''*'",
 '"-"',
 '" "',
 '"&"',
 '"^"',
 '"*"']

### File_Name: git_foospidy_payloads_other_sqli_libinjection-bypasses.txt
##### Comments: Should Remove &()o1: from start of each sentence
#### Source: https://github.com/foospidy/payloads/tree/master/other/sqli

In [37]:
path='./libinjection-bypasses.txt'

In [38]:
# read data from file

sql_lines_bypasses=[]
f = open(path, "r")
for x in f:
    sql_lines_bypasses.append(x)

In [39]:
sql_lines_bypasses[:5] # before cleaning

['&()o1: select * from users where id=1 or (\\)=1 union select 1,@@VERSION -- 1\n',
 '&(.)o: select * from users where id=1 or (\\.)=1 union select 1,@@VERSION -- 1\n',
 '&(1&1: select * from users where id=1 or (\\+)=1 or 1=1 -- 1\n',
 '&(1)o: select * from users where id=1 or (1)=1 union select 1,banner from v$version where rownum=1 -- 1\n',
 '&(1UE: select * from users where id=1 or (\\+)=1 union select 1,@@VERSION -- 1\n']

In [40]:
print(len(sql_lines_bypasses))

474


In [41]:
sql_lines_bypasses=clean_sqli_data(sql_lines_bypasses)

In [42]:
sql_lines_bypasses[:5]  # data after cleaning

['& (  ) o1: select * from users where id = 1 or  ( \\ )  = 1 union select 1,@@VERSION -- 1',
 '& ( . ) o: select * from users where id = 1 or  ( \\. )  = 1 union select 1,@@VERSION -- 1',
 '& ( 1&1: select * from users where id = 1 or  ( \\+ )  = 1 or 1 = 1 -- 1',
 '& ( 1 ) o: select * from users where id = 1 or  ( 1 )  = 1 union select 1,banner from v$version where rownum = 1 -- 1',
 '& ( 1UE: select * from users where id = 1 or  ( \\+ )  = 1 union select 1,@@VERSION -- 1']

In [43]:
# if don't want &(*)* sign in beginning of each sentence then run next code else don't

for i in range(len(sql_lines_bypasses)):
    sentence=sql_lines_bypasses[i]
    sql_lines_bypasses[i]=sentence.split(':')[1]
    

In [44]:
sql_lines_bypasses[:5]

[' select * from users where id = 1 or  ( \\ )  = 1 union select 1,@@VERSION -- 1',
 ' select * from users where id = 1 or  ( \\. )  = 1 union select 1,@@VERSION -- 1',
 ' select * from users where id = 1 or  ( \\+ )  = 1 or 1 = 1 -- 1',
 ' select * from users where id = 1 or  ( 1 )  = 1 union select 1,banner from v$version where rownum = 1 -- 1',
 ' select * from users where id = 1 or  ( \\+ )  = 1 union select 1,@@VERSION -- 1']

### File_Name: git_foospidy_payloads_owasp_sqli.txt
##### Comments: ok
#### Source: https://github.com/foospidy/payloads/tree/master/other/sqli

In [45]:
path='./data/collection/git_foospidy_payloads_owasp_sqli.txt'

In [46]:
# read data from file

sql_lines_owasp=[]
f = open(path, "r")
for x in f:
    sql_lines_owasp.append(x)

FileNotFoundError: [Errno 2] No such file or directory: './data/collection/git_foospidy_payloads_owasp_sqli.txt'

In [47]:
sql_lines_owasp[0:15] # before cleaning

[]

In [48]:
print(len(sql_lines_owasp))

0


In [49]:
sql_lines_owasp=clean_sqli_data(sql_lines_owasp)

In [50]:
sql_lines_owasp[:15] # after cleaning

[]

### File_Name: git_seclist_fuzzing_sqli_Generic-SQLi.txt
##### comments: should we remove %20 by spaces because %20 equals to space
#### Source: https://github.com/danielmiessler/SecLists/tree/master/Fuzzing/SQLi

In [51]:
path='./Generic-SQLi.txt'

In [52]:
# read data from file

sql_lines_Generic=[]
f = open(path, "r")
for x in f:
    sql_lines_Generic.append(x)

In [53]:
sql_lines_Generic[:15] # before cleaning

[")%20or%20('x'='x\n",
 '%20or%201=1\n',
 "; execute immediate 'sel' || 'ect us' || 'er'\n",
 'benchmark(10000000,MD5(1))#\n',
 'update\n',
 '";waitfor delay \'0:0:__TIME__\'--\n',
 '1) or pg_sleep(__TIME__)--\n',
 '||(elt(-3+5,bin(15),ord(10),hex(char(45))))\n',
 '"hi"") or (""a""=""a"\n',
 'delete\n',
 'like\n',
 '" or sleep(__TIME__)#\n',
 'pg_sleep(__TIME__)--\n',
 '*(|(objectclass=*))\n',
 'declare @q nvarchar (200) 0x730065006c00650063 ...\n']

In [54]:
print(len(sql_lines_Generic))

267


In [55]:
sql_lines_Generic=clean_sqli_data(sql_lines_Generic)

In [56]:
sql_lines_Generic[:15] # after cleaning

[" )  or  ( 'x' = 'x",
 ' or 1 = 1',
 "; execute immediate 'sel' || 'ect us' || 'er'",
 'benchmark ( 10000000,MD5 ( 1  )  )  #',
 'update',
 '";waitfor delay \'0:0:__TIME__\'--',
 '1 )  or pg_sleep ( __TIME__ ) --',
 '|| ( elt ( -3+5,bin ( 15 ) ,ord ( 10 ) ,hex ( char ( 45  )  )    )  )  ',
 '"hi"" )  or  ( ""a"" = ""a"',
 'delete',
 'like',
 '" or sleep ( __TIME__ ) #',
 'pg_sleep ( __TIME__ ) --',
 '* ( | ( objectclass = *  )  )  ',
 'declare @q nvarchar  ( 200 )  0x730065006c00650063 ...']

### File_name: scottparker_ml_sqli_src__trainingdata_plain.txt
##### Comments: 
removed last 22 records containing URLs
#### Source: https://github.com/tungpv98/Detect-Sql-Injection-by-Machine-Learning/tree/ebeb3287931677a3a42f11a7a08dbc13e374ff05/Sql-Injection/source/trainingdata

In [57]:

# function to remove stopwords

stop_words = set(stopwords.words('english')) 

def fun_remove_stop_words(posts):

    filtered=''
    
    for x in posts.split(' '):
        if x not in stop_words:
            filtered+=' '+x
    
    return filtered

In [58]:
path='./'
file="plain.txt"

In [59]:
#read benign data

df = pd.read_csv(os.path.join(path,file), sep='Aw3s0meSc0t7', names=['benign'], header=None, engine='python')

In [60]:
df.head()

,benign
0,Add plain text here
1,“Ne te quaesiveris extra.”
2,“Man is his own star; and the soul that can
3,"Render an honest and a perfect man,"
4,"Commands all light, all influence, all fate;"


In [61]:
plain_text=df['benign'].values  # get sentences

In [62]:
plain_text[:5]

array(['Add plain text here', '“Ne te quaesiveris extra.”',
       '“Man is his own star; and the soul that can',
       'Render an honest and a perfect man,',
       'Commands all light, all influence, all fate;'], dtype=object)

In [63]:
plain_text=plain_text[:-22] # removed last 22 records that were urls

In [64]:
len(plain_text)

3672

In [65]:
# convert from list to string

data=''
for x in plain_text:
    data+=" " + x

In [66]:
type(data)

str

In [67]:
data=fun_remove_stop_words(data)  # remove stop words
data=data.split('.')              # split sentences

In [68]:
# seperate words inside tags

for i in range(len(data)):
    data[i]=data[i].replace('<', ' <')
    data[i]=data[i].replace('>', '> ')
    data[i]=data[i].replace('=', ' = ')

In [69]:
data[:5]

['  Add plain text “Ne te quaesiveris extra',
 '” “Man star; soul Render honest perfect man, Commands light, influence, fate; Nothing falls early late',
 ' Our acts angels are, good ill, Our fatal shadows walk us still',
 '” Epilogue Beaumont Fletcher’s Honest Man’s Fortune Cast bantling rocks, Suckle she-wolf’s teat; Wintered hawk fox, Power speed hands feet',
 ' I read day verses written eminent painter original conventional']

Statistics

In [70]:
print("Benign records: %2i" %len(data))

Benign records: 3072


In [71]:
# read self created benign data

path='./benign_for_training.txt'
benign_data=[]
f = open(path, "r")
for x in f:
    benign_data.append(x)


In [72]:
# read self created sqli data

path='./sqli_for_training.txt'
sqli_data=[]
f = open(path, "r", encoding="utf8")
for x in f:
    sqli_data.append(x)


In [73]:
len(benign_data)

440

In [74]:
benign_sentence=[]
for i in benign_data:
    sentences=i.split('.')
    
    for sentence in sentences:
        benign_sentence.append(sentence)

In [75]:
len(benign_sentence)

1115

In [76]:
print("total benign data : " ,len(benign_sentence)+len(data))

total benign data :  4187


In [77]:
len(sqli_data)

290

In [78]:
1128+290

1418

In [79]:
print(f"SQL fuzzing : {len(sql_lines_fuzzing)}  camoufl4gs : {len(sql_lines_camoufl4g3)} parsed : {len(sql_lines_bypasses)} owasp : {len(sql_lines_owasp)} generic : {len(sql_lines_Generic)} \n total sql injection data : {len(sql_lines_Generic)+len(sql_lines_owasp)+len(sql_lines_bypasses)+len(sql_lines_camoufl4g3)+len(sql_lines_fuzzing)} ")

SQL fuzzing : 142  camoufl4gs : 77 parsed : 474 owasp : 0 generic : 267 
 total sql injection data : 960 


#### combination of all sqli attacks

In [80]:
all_sqli_sentence=sql_lines_owasp+sql_lines_bypasses+sql_lines_camoufl4g3+sql_lines_fuzzing+sql_lines_Generic

In [81]:
len(all_sqli_sentence)

960

In [82]:
# replace numeric values by a keyword 'numeric'
def optional_numeric_to_numeric(all_sqli_sentence):
    
    for i in range(len(all_sqli_sentence)):
        

        all_sqli_sentence[i]=all_sqli_sentence[i].replace('1 ', 'numeric')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(' 1', 'numeric')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace("'1 ", "'numeric ")
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 1'", " numeric'")
        all_sqli_sentence[i]=all_sqli_sentence[i].replace('1,', 'numeric,')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace("1\ ", 'numeric')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace("‘1", '‘numeric')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 2 ", " numeric ")
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(' 3 ', ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(' 3--', ' numeric--')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 4 ", ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 5 ", ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(' 6 ', ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 7 ", ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 8 ", ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace('1234', ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace("22", ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 8 ", ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 200 ", ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace("23 ", ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace('"1', '"numeric')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace('1"', '"numeric')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace("7659", 'numeric')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 37 ", ' numeric ')
        all_sqli_sentence[i]=all_sqli_sentence[i].replace(" 45 ", ' numeric ')
    
    return all_sqli_sentence


In [83]:
all_sqli_sentence

[' select * from users where id = 1 or  ( \\ )  = 1 union select 1,@@VERSION -- 1',
 ' select * from users where id = 1 or  ( \\. )  = 1 union select 1,@@VERSION -- 1',
 ' select * from users where id = 1 or  ( \\+ )  = 1 or 1 = 1 -- 1',
 ' select * from users where id = 1 or  ( 1 )  = 1 union select 1,banner from v$version where rownum = 1 -- 1',
 ' select * from users where id = 1 or  ( \\+ )  = 1 union select 1,@@VERSION -- 1',
 ' select * from users where id = 1 or  ( $+ )  = 1 or 1 = 1 -- 1',
 ' select * from users where id = 1 or  ( $+ )  = 1 union select 1,@@VERSION -- 1',
 ' select * from users where id = 1 or \\.<\\ or 1 = 1 -- 1',
 ' select * from users where id = 1 or \\.<1 or 1 = 1 -- 1',
 ' select * from users where id = 1 or \\.<1 union select 1,@@VERSION -- 1',
 ' select * from users where id = 1 or \\.<\\ union select 1,@@VERSION -- 1',
 ' select * from users where id = 1 or \\.<$ or 1 = 1 -- 1',
 ' select * from users where id = 1 or \\.<$ union select 1,@@VERSION -- 1

#### Combination of benign an attack data

In [84]:
import pandas as pd

In [85]:
# give labels to sql data

values=[]
for i in all_sqli_sentence:
    values.append((i,1))

In [86]:
# give labels to benign data

for i in data:
    values.append((i,0))

In [87]:
len(all_sqli_sentence)+len(data)

4032

In [88]:
len(values)

4032

In [89]:
for i in benign_sentence:
    values.append((i,0))

In [90]:
len(values)

5147

In [91]:
for i in sqli_data:
    values.append((i,1))

In [92]:
len(values)

5437

In [93]:
values[1]

(' select * from users where id = 1 or  ( \\. )  = 1 union select 1,@@VERSION -- 1',
 1)

In [94]:
# convert to dataframe

df=pd.DataFrame(values,columns=['Sentence','Label'])



In [95]:
df.head()

,Sentence,Label
0,select * from users where id = 1 or ( \ ) =...,1
1,select * from users where id = 1 or ( \. ) ...,1
2,select * from users where id = 1 or ( \+ ) ...,1
3,select * from users where id = 1 or ( 1 ) =...,1
4,select * from users where id = 1 or ( \+ ) ...,1


### Save data as csv

In [96]:
df.to_csv('sqli.csv', index=False, encoding='utf-16')

In [97]:
df=pd.read_csv('sqli.csv',encoding='utf-16')

In [98]:
# vectorization of data

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer( min_df=2, max_df=0.7, max_features=4096, stop_words=stopwords.words('english'))
posts = vectorizer.fit_transform(df['Sentence'].values.astype('U')).toarray()



In [99]:
posts.shape

(5437, 4096)

In [100]:
posts.shape=(5437,64,64,1)

In [101]:
posts.shape

(5437, 64, 64, 1)

In [102]:
X=posts

In [103]:
y=df['Label']

In [104]:
from sklearn.model_selection import train_test_split

In [105]:
# split train test data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [106]:
trainX=X_train.copy()
trainX.shape=(X_train.shape[0],trainX.shape[1]*trainX.shape[2])
testX=X_test.copy()
testX.shape=(testX.shape[0],testX.shape[1]*testX.shape[2])


#### Naive Bayes

In [107]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()

gnb.fit(trainX, y_train)

pred_gnb = gnb.predict(testX)

#### Support Vector Machine

In [108]:
# SVM
from sklearn.svm import SVC
clf = SVC(gamma='auto')
clf.fit(trainX, y_train)
pred_svm=clf.predict(testX)

#### K Nearest Neighbor

In [109]:
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(trainX, y_train)

pred_knn = neigh.predict(testX)

#### Decision Tree

In [110]:
from sklearn import tree

dt = tree.DecisionTreeClassifier()
dt = dt.fit(trainX, y_train)

pred_dt = dt.predict(testX)

In [111]:
X_train.shape

(4349, 64, 64, 1)

#### CNN

In [112]:
import tensorflow as tf
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.wrappers.scikit_learn import KerasClassifier

In [113]:
model=tf.keras.models.Sequential([
    
    tf.keras.layers.Conv2D(64, (3,3), activation=tf.nn.relu, input_shape=(64,64,1)),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(128,(3,3), activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(256,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(64, activation=tf.nn.relu),
    tf.keras.layers.Dense(1, activation='sigmoid')
])



In [114]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 64)        640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 256)       295168    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 256)         0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0

In [115]:
# import tensorflow as tf

# class myCallback(tf.keras.callbacks.Callback):
#     def on_epoch_end(self, epoch, logs={}):
#         if(logs.get('acc')>0.98):
#             print( "Reached 99.8% accuracy so cancelling training!")
#             self.model.stop_training=True
# callbacks = myCallback()

In [116]:
classifier_nn = model.fit(X_train,y_train,
                    epochs=10,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=128)


Epoch 1/10
34/34 [==============================] - 43s 1s/step - loss: 0.3131 - accuracy: 0.8393 - val_loss: 0.2022 - val_accuracy: 0.8934
Epoch 2/10
34/34 [==============================] - 39s 1s/step - loss: 0.1718 - accuracy: 0.9071 - val_loss: 0.1714 - val_accuracy: 0.9072
Epoch 3/10
34/34 [==============================] - 41s 1s/step - loss: 0.1404 - accuracy: 0.9303 - val_loss: 0.1616 - val_accuracy: 0.9173
Epoch 4/10
34/34 [==============================] - 41s 1s/step - loss: 0.1169 - accuracy: 0.9402 - val_loss: 0.1554 - val_accuracy: 0.9182
Epoch 5/10
34/34 [==============================] - 42s 1s/step - loss: 0.1032 - accuracy: 0.9446 - val_loss: 0.1572 - val_accuracy: 0.9182
Epoch 6/10
34/34 [==============================] - 44s 1s/step - loss: 0.0904 - accuracy: 0.9494 - val_loss: 0.1733 - val_accuracy: 0.9246
Epoch 7/10
34/34 [==============================] - 43s 1s/step - loss: 0.0931 - accuracy: 0.9499 - val_loss: 0.1564 - val_accuracy: 0.9265
Epoch 8/10
34/34 [==

In [117]:
pred=model.predict(X_test)

In [118]:
for i in range(len(pred)):
    if pred[i]>0.5:
        pred[i]=1
    elif pred[i]<=0.5:
        pred[i]=0

In [119]:
from sklearn.metrics import accuracy_score

In [120]:
accuracy_score(y_test,pred)

0.9227941176470589

In [121]:
for i,j in zip(y_test,pred):
    print(i==j)

[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[False]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[False]
[ True]
[ True]
[False]
[ True]
[False]
[ True]
[False]
[ True]
[ True]
[ True]
[ True]
[ True]
[False]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[False]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[False]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[False]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[False]
[ True]
[False]
[ True]
[ True]
[ True]
[ True]
[False]
[ True]
[ True]
[ True]
[False]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]
[ True]


#### Saving Model

In [122]:
from keras.models import load_model
import pickle

model.save('my_model_cnn.h5')
with open('vectorizer_cnn', 'wb') as fin:
    pickle.dump(vectorizer, fin)


### Confusion Matrix
#### Assumption:
1. SQLI attack type is positive

In [123]:
def accuracy_function(tp,tn,fp,fn):
    
    accuracy = (tp+tn) / (tp+tn+fp+fn)
    
    return accuracy

In [124]:
def precision_function(tp,fp):
    
    precision = tp / (tp+fp)
    
    return precision

In [125]:
def recall_function(tp,fn):
    
    recall=tp / (tp+fn)
    
    return recall

In [126]:
def confusion_matrix(truth,predicted):
    
    true_positive = 0
    true_negative = 0
    false_positive = 0
    false_negative = 0
    
    for true,pred in zip(truth,predicted):
        
        if true == 1:
            if pred == true:
                true_positive += 1
            elif pred != true:
                false_negative += 1

        elif true == 0:
            if pred == true:
                true_negative += 1
            elif pred != true:
                false_positive += 1
            
    accuracy=accuracy_function(true_positive, true_negative, false_positive, false_negative)
    precision=precision_function(true_positive, false_positive)
    recall=recall_function(true_positive, false_negative)
    
    return (accuracy,
            precision,
           recall)

In [127]:
accuracy,precision,recall=confusion_matrix(y_test,pred)

In [128]:
print(" For CNN \n Accuracy : {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))

 For CNN 
 Accuracy : 0.9227941176470589 
 Precision : 0.7966666666666666 
 Recall : 0.9122137404580153


In [129]:
accuracy,precision,recall=confusion_matrix(y_test,pred_gnb)
print(" For Naive Bayes Accuracy : {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))

 For Naive Bayes Accuracy : 0.9494485294117647 
 Precision : 0.8371335504885994 
 Recall : 0.9809160305343512


In [130]:
accuracy,precision,recall=confusion_matrix(y_test,pred_svm)
print(" For SVM Accuracy : {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))

 For SVM Accuracy : 0.8134191176470589 
 Precision : 1.0 
 Recall : 0.22519083969465647


In [131]:
accuracy,precision,recall=confusion_matrix(y_test,pred_knn)
print(" For KNN Accuracy : {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))

 For KNN Accuracy : 0.8805147058823529 
 Precision : 0.7481203007518797 
 Recall : 0.7595419847328244


In [132]:
accuracy,precision,recall=confusion_matrix(y_test,pred_dt)
print(" For Decision Tree Accuracy : {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))

 For Decision Tree Accuracy : 0.8492647058823529 
 Precision : 0.6200980392156863 
 Recall : 0.9656488549618321


### Classifying user data

     This cell is also in file user_Data_predict.py No need to run this cell

In [133]:

import keras
from keras.models import load_model
import pickle


mymodel = tf.keras.models.load_model('my_model_cnn.h5')
myvectorizer = pickle.load(open("vectorizer_cnn", 'rb'))




def clean_data(input_val):

    input_val=input_val.replace('\n', '')
    input_val=input_val.replace('%20', ' ')
    input_val=input_val.replace('=', ' = ')
    input_val=input_val.replace('((', ' (( ')
    input_val=input_val.replace('))', ' )) ')
    input_val=input_val.replace('(', ' ( ')
    input_val=input_val.replace(')', ' ) ')
    input_val=input_val.replace('1 ', 'numeric')
    input_val=input_val.replace(' 1', 'numeric')
    input_val=input_val.replace("'1 ", "'numeric ")
    input_val=input_val.replace(" 1'", " numeric'")
    input_val=input_val.replace('1,', 'numeric,')
    input_val=input_val.replace(" 2 ", " numeric ")
    input_val=input_val.replace(' 3 ', ' numeric ')
    input_val=input_val.replace(' 3--', ' numeric--')
    input_val=input_val.replace(" 4 ", ' numeric ')
    input_val=input_val.replace(" 5 ", ' numeric ')
    input_val=input_val.replace(' 6 ', ' numeric ')
    input_val=input_val.replace(" 7 ", ' numeric ')
    input_val=input_val.replace(" 8 ", ' numeric ')
    input_val=input_val.replace('1234', ' numeric ')
    input_val=input_val.replace("22", ' numeric ')
    input_val=input_val.replace(" 8 ", ' numeric ')
    input_val=input_val.replace(" 200 ", ' numeric ')
    input_val=input_val.replace("23 ", ' numeric ')
    input_val=input_val.replace('"1', '"numeric')
    input_val=input_val.replace('1"', '"numeric')
    input_val=input_val.replace("7659", 'numeric')
    input_val=input_val.replace(" 37 ", ' numeric ')
    input_val=input_val.replace(" 45 ", ' numeric ')

    return input_val








def predict_sqli_attack():
    
    repeat=True
    
    beautify=''
    for i in range(20):
        beautify+= "="

    print(beautify) 
    input_val=input("Give me some data to work on : ")
    print(beautify)

    
    if input_val== '0':
        repeat=False
    
    

    input_val=clean_data(input_val)
    input_val=[input_val]



    input_val=myvectorizer.transform(input_val).toarray()
    
    input_val.shape=(1,64,64,1)

    result=mymodel.predict(input_val)


    print(beautify)
    
    
    if repeat == True:
        
        if result>0.5:
            print("ALERT :::: This can be SQL injection")


        elif result<=0.5:
            print("It seems to be safe")
            
        print(beautify)
            
        predict_sqli_attack()
            
    elif repeat == False:
        print( " Good Bye ")

 



In [134]:
# check for sql injection cheat sheet





predict_sqli_attack()

Give me some data to work on : cekidot baju ini 111
ALERT :::: This can be SQL injection
Give me some data to work on : maybe
ALERT :::: This can be SQL injection
Give me some data to work on : select
ALERT :::: This can be SQL injection
Give me some data to work on : 
ALERT :::: This can be SQL injection
Give me some data to work on : 0
 Good Bye 


In [136]:
# check for normal data





predict_sqli_attack()

Give me some data to work on : cekidot baju ini 111
ALERT :::: This can be SQL injection
Give me some data to work on : test
It seems to be safe
Give me some data to work on : maybe
ALERT :::: This can be SQL injection
Give me some data to work on : i will try
ALERT :::: This can be SQL injection
Give me some data to work on : example.com
It seems to be safe
Give me some data to work on : youtube.com
It seems to be safe
Give me some data to work on : youtube.com/?id=1
It seems to be safe
Give me some data to work on : cek
ALERT :::: This can be SQL injection
Give me some data to work on : 0
 Good Bye 


In [137]:
predict_sqli_attack()


Give me some data to work on : cek
ALERT :::: This can be SQL injection
Give me some data to work on : done
It seems to be safe
Give me some data to work on : server is up
It seems to be safe
Give me some data to work on : select
ALERT :::: This can be SQL injection
Give me some data to work on : mywebsite.com/?user='matt'
It seems to be safe
Give me some data to work on : mywebsite.com/?user='matt'' or 1=1
It seems to be safe
Give me some data to work on : mywebsite.com/?user='matt''
It seems to be safe
Give me some data to work on : mywebsite.com/?website
It seems to be safe
Give me some data to work on : union
It seems to be safe
Give me some data to work on : cek
ALERT :::: This can be SQL injection
Give me some data to work on : index
ALERT :::: This can be SQL injection
Give me some data to work on : try
ALERT :::: This can be SQL injection
Give me some data to work on : try the new drink
It seems to be safe
Give me some data to work on : try
ALERT :::: This can be SQL injection
